# 要約 
このJupyter Notebookは、Kaggleの「LMSYS - Chatbot Arena 人間による好み予測チャレンジ」において、ユーザーの好みを予測する問題に取り組んでいます。具体的には、複数の大規模言語モデル（LLM）からの応答に対して、どの応答がより好まれるかを予測するためのモデルを構築しています。

### 主な内容と手法

1. **データの前処理**: 
   - `pandas`を用いてテストデータを読み込み、各プロンプトと応答のテキストを処理する関数を定義します。この処理では、特定の文字を除去し、テキストをクレンジングしてから新しいカラムに格納しています。

2. **カスタムデータセットの作成**:
   - `torch.utils.data.Dataset`を継承した`Senmamtic_news`クラスを作成し、プロンプトと応答をトークナイズ（BERTトークナイザーを使用）して、モデルへの入力形式に整えています。
   - 応答が長すぎる場合には適切にトリミングを行う処理を実装しています。

3. **データローダーの取得**:
   - データローダーは、バッチ処理およびデータのシャッフルを行うための関数`get_semantic_data_loader`を通じて取得され、トレーニングとテストの両方のモードに対応しています。

4. **モデルの定義と推論**:
   - `BertPET`クラスとして、事前学習済みのBERTモデルを利用したマスク言語モデルを定義しています。
   - モデルを使用して推論を行い、得られた確率的出力を`inference`関数で計算します。

5. **出力結果の生成**:
   - 最終的に、テストデータに対する予測結果を保存するために、提出用のCSVファイル`submission.csv`を作成しています。出力フォーマットはコンペティションで要求される形式に従っています。

### 使用ライブラリ
- `transformers`（BERTモデル用）
- `torch`（PyTorch）
- `pandas`（データ処理）
- `tqdm`（進捗表示）

このノートブックは、言語モデルとユーザーの選好を結びつけるための強力な機械学習技術を活用しており、チャットボットの応答の質を向上させるための具体的なアプローチを示しています。

---


# 用語概説 
以下に、初心者がつまずきそうな専門用語の簡単な解説を挙げます。特に、実務経験が少ない方や、このノートブック特有のドメイン知識に焦点を当てています。

1. **トークナイジング (Tokenization)**:
   - 自然言語処理(NLP)において、文章を構成する単語やサブワードに分割するプロセスです。モデルは整数の配列でデータを扱うため、テキストをトークンと呼ばれる小さな単位に変換する必要があります。

2. **マスクトークン (Mask Token)**:
   - 言語モデルの学習において、特定の単語を予測するためにその単語を隠すために使われる特殊なトークンのことです。モデルはこのマスクされた部分を推測するように訓練されます。BERTなどのモデルで一般的に使用されます。

3. **パディング (Padding)**:
   - 異なる長さのシーケンス（例: トークンのリスト）を固定の長さに揃えるために、特別なトークン（通常はゼロトークン）を追加するプロセスです。これにより、バッチ処理が可能になります。

4. **アテンションマスク (Attention Mask)**:
   - トランスフォーマー系のモデルで使用され、どのトークンに注意を向けるかを示すためのマスクです。通常、トークンが存在する場合は1、存在しない場合は0の値を持ちます。

5. **ロジット (Logits)**:
   - モデルの出力層から得られる、各クラス（またはトークン）に対する未正規化のスコアのことです。これをソフトマックス関数に通すことで、クラスの確率に変換できます。

6. **データローダー (DataLoader)**:
   - PyTorchにおけるデータ操作のためのツールで、データセットをバッチ処理に分けて効率良く読み込むためのクラスです。データのシャッフルや並行処理も可能です。

7. **Masked Language Model (MLM)**:
   - 文中の特定のトークンを隠し、そのトークンを予測することを目的とした言語モデルです。BERTなどのトランスフォーマーモデルがこのアプローチを使用しています。

8. **注意機構 (Attention Mechanism)**:
   - 特にトランスフォーマーにおいてよく用いられる手法で、入力シーケンスの異なる部分に異なる重みを与えて、モデルが文脈に基づいて情報を処理することを可能にします。

9. **ファインチューニング (Fine-tuning)**:
   - 事前学習済みのモデルを特定のタスク向けに再調整するプロセスです。一般的なデータセットで学習したモデルを、特定のデータセットに合わせて微調整します。

10. **トランスフォーマー (Transformer)**:
    - 自然言語処理において非常に人気のあるモデルアーキテクチャで、自己注意機構を利用しています。BERTやGPTなど、多くの最新のNLPモデルはこのアーキテクチャに基づいています。

これらの用語は、ノートブックのコードや処理フローと関連しており、特に初学者や実務経験が浅い方によくつまずくポイントです。

---


In [ ]:
from itertools import zip_longest
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from transformers import BertTokenizer, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch
import pandas as pd
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

# データを処理する関数
def process(example):
    # プロンプトの文をリストに変換し、特定の文字を取り除く
    sentences = [s.strip('"').replace('[MASK]','') for s in example['prompt'].strip('[]').split('","')]
    # 応答Aの文をリストに変換し、特定の文字を取り除く
    sentences_a = [s.strip('"').replace('[MASK]','') for s in example['response_a'].strip('[]').split('","')]
    # 応答Bの文をリストに変換し、特定の文字を取り除く
    sentences_b = [s.strip('"').replace('[MASK]','') for s in example['response_b'].strip('[]').split('","')]
    # プロンプトと応答Aの文を組み合わせ、空の文は除外
    texts_a = [p for pair in zip_longest(sentences, sentences_a, fillvalue='') for p in pair if p]
    # プロンプトと応答Bの文を組み合わせ、空の文は除外
    texts_b = [p for pair in zip_longest(sentences, sentences_b, fillvalue='') for p in pair if p]
    # プロセス結果をデータシリーズとして返す
    return pd.Series([' '.join(sentences), ' '.join(sentences_a), ' '.join(sentences_b), '\n'.join(texts_a), '\n'.join(texts_b)], index=['prompt', 'response_a', 'response_b', 'text_a','text_b'])

# データフレームに対して処理関数を適用
test[['prompt', 'response_a', 'response_b', 'text_a','text_b']] = test.apply(process, axis=1)
# 欠損値を含む行を削除
test = test.dropna()

# 自作データセットクラス
class Senmamtic_news(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
        self.sep_token = tokenizer.sep_token
        self.mask_token = tokenizer.mask_token

        prompt_new = []
        # プロンプトのトークナイジング
        tk0 = tqdm(data['prompt'].fillna("").values, total=len(data))
        for text in tk0:
            length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
            # プロンプトが長すぎる場合、前半と後半を結合する
            if (length > 512):
                text = tokenizer.convert_tokens_to_string(
                    tokenizer.tokenize(text)[:256] + tokenizer.tokenize(text)[-256:])
            prompt_new.append(text)
        
        print(f'== response_a ==')
        response_a_new = []
        # 応答Aのトークナイジング
        tk0 = tqdm(data['response_a'].fillna("").values, total=len(data))
        for text in tk0:
            length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
            if (length > 512):
                text = tokenizer.convert_tokens_to_string(
                    tokenizer.tokenize(text)[:256] + tokenizer.tokenize(text)[-256:])
            response_a_new.append(text)

        print(f'== response_b ==')
        response_b_new = []
        # 応答Bのトークナイジング
        tk0 = tqdm(data['response_b'].fillna("").values, total=len(data))
        for text in tk0:
            length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
            if (length > 512):
                text = tokenizer.convert_tokens_to_string(
                    tokenizer.tokenize(text)[:256] + tokenizer.tokenize(text)[-256:])
            response_b_new.append(text)
        
        # 新しいプロンプトと応答をデータフレームに格納
        self.data['prompt'] = prompt_new
        self.data['response_a'] = response_a_new
        self.data['response_b'] = response_b_new

    def __len__(self):
        # データの長さを返す
        return len(self.data)

    def __getitem__(self, idx):
        # データポイントの取得
        prompt = self.data['prompt'][idx]
        response_a = self.data['response_a'][idx]
        response_b = self.data['response_b'][idx]
        system_prompt = f"""{prompt}:
                        Response A: {response_a}
                        Response B: {response_b}
                        Which is better? Choose 'A', 'B', or 'both'.
""" + self.mask_token

        # モデルの入力用にトークン化
        inputs = self.tokenizer(system_prompt, truncation=True, max_length=1600)
        input_ids  = torch.tensor(inputs['input_ids'], dtype=torch.long)
        mask_index = torch.where(input_ids == self.tokenizer.mask_token_id)[0]

        return {
            'input_ids': input_ids,
            'attention_mask': torch.tensor(inputs['attention_mask'], dtype=torch.long),
            'mask_index': mask_index
        }

# データをバッチにするためのカスタム関数
def collate_fn_semantic(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    mask_index = [item['mask_index'] for item in batch]

    # パディングを施し、バッチ処理
    input_ids = pad_sequence(input_ids, batch_first=True)
    attention_mask = pad_sequence(attention_mask, batch_first=True,)
    mask_index = torch.stack(mask_index)
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'mask_index': mask_index
    }

# データローダーを取得する関数
def get_semantic_data_loader(batch_size=32, mode='train', shuffle=True):
    tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/deberta-small/deberta')
    if mode == 'train':
        dataset = Senmamtic_news(train, tokenizer)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_fn_semantic)
        return dataloader
    else:
        dataset = Senmamtic_news(test, tokenizer)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn_semantic, drop_last=False)
        return dataloader

import torch
from torch import nn
from transformers import AutoModelForMaskedLM, AutoConfig
from tqdm import tqdm

# モデルクラスの定義
class BertPET(nn.Module):
    def __init__(self, model_path):
        super().__init__()
        self.bert = AutoModelForMaskedLM.from_pretrained(model_path)
        self.config = AutoConfig.from_pretrained(model_path)

    def forward(self, input_ids, attention_mask, mask_index, labels=None, return_logits=False):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask).logits
        mask_index = mask_index.unsqueeze(-1).expand(-1, -1, output.size(-1))
        output = torch.gather(output, 1, mask_index).squeeze(1)
        if return_logits:
            # 特定インデックスのlogitsを返す
            logits = output[:,[336, 732, 462]]
            return logits

# 推論用の関数
def inference(model, dataloader, device):
    model.eval()
    softmax = nn.Softmax(dim=-1)
    all_probs = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Inference", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            mask_index = batch['mask_index'].to(device)

            logits = model(input_ids=input_ids, attention_mask=attention_mask, mask_index=mask_index, return_logits=True)
            probs = softmax(logits)
            probs_list = probs.cpu().numpy().tolist()
            for i in probs_list:
                all_probs.append(i)

    return all_probs

# デバイス設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# モデルと重みをロード
model_path = '/kaggle/input/deberta-small/deberta'  # 事前学習済みモデルのパス
checkpoint_path = '/kaggle/input/deberta-v3/model_checkpoint_epoch_2.pt'  # 最良重みファイルのパス
model = BertPET(model_path)
model.load_state_dict(torch.load(checkpoint_path, map_location=device))
model.to(device)

# テストデータを取得
test_loader = get_semantic_data_loader(2, mode='test')

# 推論を実施
test_probs = inference(model, test_loader, device)

# 提出ファイルの作成
sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')
test[['winner_model_a', 'winner_model_b', 'winner_tie']] = test_probs
sample_sub = sample_sub[['id']].merge(test[['id', 'winner_model_a', 'winner_model_b', 'winner_tie']], on='id', how='left')
sample_sub.to_csv('submission.csv', index=False)
display(sample_sub.head())